# 3D Visualization

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

import torch.optim as optim
from rockgan.architecture import *
from rockgan.utils import *

import tifffile as tiff
from skimage import transform
from skimage.measure import block_reduce
from scipy.spatial.transform import Rotation as R

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

DEVICE = torch.device('cuda')

## Training Data

In [3]:
dataset = tiff.imread('../data/berea.tif')

In [5]:
downsampled = block_reduce(dataset, block_size=(2, 2, 2), func=np.mean)

# Verify the shapes
print(dataset.shape)       # Prints: (60, 60, 60)
print(downsampled.shape) # Prints: (30, 30, 30)

(400, 400, 400)
(200, 200, 200)


In [6]:
filename = "../figures/videos/real_rock_rotating.mp4"

grid = pv.UniformGrid(dims=(201,201,201))
grid.cell_data["values"] = downsampled[:200,:200,:200].flatten() 
mesh = grid.clip_box()

plotter = pv.Plotter()
# Open a movie file
plotter.open_movie(filename)

# Add initial mesh
plotter.add_mesh(mesh, cmap='gray', show_scalar_bar=False)
# Add outline for reference
plotter.add_mesh(mesh.outline_corners())

plotter.show(auto_close=False)  # only necessary for an off-screen movie

# Run through each frame
plotter.write_frame()  # write initial data

# Get center of the mesh and convert it to numpy array
center = np.array(mesh.center)

# Rotate mesh on each frame
for i in range(360):
    # Translate to origin
    mesh.translate(-center)

    # Rotate and translate back
    rot = R.from_euler('z', 1, degrees=True)
    mesh.points = rot.apply(mesh.points)

    # Translate back to original center
    mesh.translate(center)

    # plotter.add_text(f"Iteration: {i}", name='time-label', viewport=[0.1, 0.1])
    plotter.add_text('Field Sample', viewport=[0.1, 0.1])
    plotter.write_frame()  # Write this frame

# Be sure to close the plotter when finished
plotter.close()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

/home/corralma/miniconda3/envs/rockgan/lib/python3.10/site-packages/pyvista/core/pointset.py:205: PyvistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyvistaDeprecationWarning)


## RockGAN samples

In [21]:
# Training hyperparameters
LEARNING_RATE = 1e-3
BATCH_SIZE = 32
Z_DIM = 16
# fixed noise for display
fixed_noise = torch.randn(BATCH_SIZE,1, Z_DIM, Z_DIM, Z_DIM).to(DEVICE)
# generator
gen = Generator(in_channel=1, out_channel=1).to(DEVICE)
# optimizer
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))

In [34]:
#Loading generator per epochs
load_checkpoint("../checkpoints/generator/generator_poro_95_v4.pt", \
                model=gen, optimizer=opt_gen, lr=1e-3)
RockGAN_images = np.round(gen(fixed_noise).detach().cpu().numpy().reshape(BATCH_SIZE,128,128,128))

=> Loading checkpoint


In [38]:
filename = "../figures/videos/generated_rock_1_rotating.mp4"

grid = pv.UniformGrid(dims=(129,129,129))
grid.cell_data["values"] = RockGAN_images[10].flatten() 
mesh = grid.clip_box()

plotter = pv.Plotter()
# Open a movie file
plotter.open_movie(filename)

# Add initial mesh
plotter.add_mesh(mesh, cmap='gray', show_scalar_bar=False)
# Add outline for reference
plotter.add_mesh(mesh.outline_corners())

plotter.show(auto_close=False)  # only necessary for an off-screen movie

# Run through each frame
plotter.write_frame()  # write initial data

# Get center of the mesh and convert it to numpy array
center = np.array(mesh.center)

# Rotate mesh on each frame
for i in range(360):
    # Translate to origin
    mesh.translate(-center)

    # Rotate and translate back
    rot = R.from_euler('z', 1, degrees=True)
    mesh.points = rot.apply(mesh.points)

    # Translate back to original center
    mesh.translate(center)

    # plotter.add_text(f"Iteration: {i}", name='time-label', viewport=[0.1, 0.1])
    plotter.add_text('Generated Sample N=1', viewport=[0.1, 0.1])
    plotter.write_frame()  # Write this frame

# Be sure to close the plotter when finished
plotter.close()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)